In [1]:
import uproot
import numpy as np
import awkward as ak
import math as m
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep as hep
import os
import numba as nb
import yaml
import boost_histogram as bh
from coffea import hist as chist
from coffea.nanoevents.methods import vector
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema, TreeMakerSchema

import sys
import time

from coffea.util import save
from coffea.util import load

from yty_hist_class import yty_histplot

In [2]:
import xgboost as xgb
import pandas as pd 
from xgboost import XGBRegressor
from xgboost import plot_importance
from xgboost import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
out_dir = 'array_for_training'

seed = 6
test_size = 0.33

data_list_training = []
data_list_test = []

signal_list_training = []
signal_list_test = []

ndata_nonp_events = 0

for isdata in ['data','mc']:
# for isdata in ['mc']:
# for isdata in ['data']:
    nevents = 0
# for isdata in ['mc']:
# for isdata in ['data']:
#     isdata = 'data'#'data' or 'mc'
    if isdata=='data':
        Data_merge = [
                    'DoubleMuon_Run2018A','DoubleMuon_Run2018B','DoubleMuon_Run2018C','DoubleMuon_Run2018D',
                     'EGamma_Run2018A','EGamma_Run2018B','EGamma_Run2018C','EGamma_Run2018D',
                     'SingleMuon_Run2018A','SingleMuon_Run2018B','SingleMuon_Run2018C','SingleMuon_Run2018D',
                     'MuonEG_Run2018A','MuonEG_Run2018B','MuonEG_Run2018C','MuonEG_Run2018D'
                     ]
        PROCESSES= [Data_merge]

        with open(f"/data/pubfs/tyyang99/jupyter_files/data_2018_nanov7.yaml", 'r') as f:
            mc_yaml = yaml.load(f, Loader=yaml.FullLoader)
    elif isdata=='mc':
        WW_polar = ['WpWpJJ_LL_polarization_EWK','WpWpJJ_TL_polarization_EWK','WpWpJJ_TT_polarization_EWK']
        WW_merge = ['WpWpJJ_EWK', 'WpWpJJ_QCD']
        WZ_EWK = ['WLLJJ_EWK']
        WZ_merge = ['WZTo3LNu_0Jets_MLL_4to50','WZTo3LNu_1Jets_MLL_4to50','WZTo3LNu_2Jets_MLL_4to50','WZTo3LNu_3Jets_MLL_4to50',
                   'WZTo3LNu_0Jets_MLL_50','WZTo3LNu_1Jets_MLL_50','WZTo3LNu_2Jets_MLL_50','WZTo3LNu_3Jets_MLL_50']
        ZZ_merge = ['ZZJJTo4L_EWK','ZZJJTo4L_QCD','ZZJJTo4L_int','ggZZ_2e2mu',
                   'ggZZ_2e2tau','ggZZ_2mu2tau','ggZZ_4e','ggZZ_4mu','ggZZ_4tau','ggZZ_4tau_ext']
        TVX_merge = ['TTGJets','TTZToQQ','TTZToQQ_ext','TTZToLLNuNu_M_10',
                     'TTWJetsToQQ','TTWJetsToLNu','tZq']
        VG_merge = ['Zgamma_EWK','Zgamma_EWK_ext','Wgamma_EWK','Wgamma_int']
        WS_merge = ['WWTo2L2Nu',
                    'ggWW_ee','ggWW_em','ggWW_et','ggWW_me','ggWW_mm','ggWW_mt','ggWW_te','ggWW_tm','ggWW_tt',
                    'TTTo2L2Nu','ST_tW_top','ST_tW_antitop',
                    'DYJetsToLL_M50','DYJets_M10to50','DYJets_M10to50_ext','ggh_ww','ggh_zz','ggh_tautau',
                    'VBF_HToZZTo4L','VBFHToWWTo2L2Nu','VBFHToTauTau',
                    'ttHToNonbb','VHToNonbb']
        Other_merge = ['WW_DS','WWW','WWZ','WZZ','ZZZ','WWG']

        PROCESSES = [WW_polar, WW_merge, WZ_EWK, WZ_merge, ZZ_merge, TVX_merge, VG_merge, WS_merge, Other_merge]
#         PROCESSES = [WW_polar]
#         PROCESSES = [WS_merge]
        with open(f"/data/pubfs/tyyang99/jupyter_files/changexs_datasets.yaml", 'r') as f:
            mc_yaml = yaml.load(f, Loader=yaml.FullLoader)
    else:
        print('!!!Wrong data type!!!')
    if isdata=='data':
        for i,process_list in enumerate(PROCESSES):
            for process in process_list:
                try:
                    arrays = ak.from_parquet(f'{out_dir}_nonprompt/{process}.parquet')
                except:
                    print(f'{process} no nonprompt events')
                    continue
                print(process, len(arrays), np.sum(arrays['weight'])/len(arrays))
                ndata_nonp_events += np.sum(arrays['weight'])
        print(ndata_nonp_events)
    if isdata=='mc':
        for i,process_list in enumerate(PROCESSES):
            for process in process_list:
                if process.endswith('_ext'):
                    continue
                else:
                    try:
                        arrays = ak.from_parquet(f'{out_dir}/{process}.parquet')
                    except:
                        print(f'{process} no events')
                        continue
                    print(process, len(arrays), np.sum(arrays['weight'])/len(arrays))
                    if process in WW_polar or process=='WpWpJJ_EWK':
                        signal = ak.ones_like(arrays['weight'])
                    else:
                        signal = ak.zeros_like(arrays['weight'])
                    try:
                        arrays_training, arrays_test, signal_training, signal_test = train_test_split(arrays, signal, test_size=test_size, train_size=(1-test_size), random_state=seed)
                    except:
                        arrays_training, arrays_test, signal_training, signal_test = arrays, [], signal, []
                    data_list_training.append(arrays_training)
                    data_list_test.append(arrays_test)

                    signal_list_training.append(signal_training)
                    signal_list_test.append(signal_test)

                    if process=='TTTo2L2Nu':
                        arrays = ak.from_parquet(f'{out_dir}_opposite-sign/{process}.parquet')
                        print(process+' opposite sign', len(arrays), np.sum(arrays['weight'])/len(arrays))
                        signal = ak.zeros_like(arrays['weight'])
                        arrays['weight'] = ndata_nonp_events/np.sum(arrays['weight'])*arrays['weight']
                        print(process+' opposite sign', len(arrays), np.sum(arrays['weight'])/len(arrays))
                        try:
                            arrays_training, arrays_test, signal_training, signal_test = train_test_split(arrays, signal, test_size=test_size, train_size=(1-test_size), random_state=seed)
                        except:
                            arrays_training, arrays_test, signal_training, signal_test = arrays, [], signal, []
                
        All_data_training = ak.concatenate(data_list_training)
        All_data_test = ak.concatenate(data_list_test)

        All_data_training['signal'] = ak.concatenate(signal_list_training)
        All_data_test['signal'] = ak.concatenate(signal_list_test)

        weight_boost_training = np.sum(All_data_training['weight'][All_data_training['signal']==0])/np.sum(All_data_training['weight'][All_data_training['signal']==1])
        All_data_training['training_weight'] = 1*(All_data_training['signal']==0)*All_data_training['weight'] + weight_boost_training*(All_data_training['signal']==1)*(All_data_training['weight'])
        pd_data_training = ak.to_pandas(All_data_training)
        pd_data_test = ak.to_pandas(All_data_test)

DoubleMuon_Run2018A 54 0.3535058817488473
DoubleMuon_Run2018B 19 0.5350714123903092
DoubleMuon_Run2018C 26 0.4135009820535478
DoubleMuon_Run2018D 114 0.3001385363192895
EGamma_Run2018A 16 0.4299506605012088
EGamma_Run2018B 10 0.3616682172762589
EGamma_Run2018C 7 0.3633864278088801
EGamma_Run2018D 60 0.32892646303771234
SingleMuon_Run2018A 23 0.3667182398951652
SingleMuon_Run2018B 10 0.3147448074150042
SingleMuon_Run2018C 12 0.29628172297297034
SingleMuon_Run2018D 68 0.3027337848524814
MuonEG_Run2018A 45 0.357718353070879
MuonEG_Run2018B 26 0.2649956716266408
MuonEG_Run2018C 27 0.3330359469501007
MuonEG_Run2018D 112 0.38154614142253235
217.4332760361935
WpWpJJ_LL_polarization_EWK 120203 6.833796911618149e-05
WpWpJJ_TL_polarization_EWK 135344 0.00032310943003734636
WpWpJJ_TT_polarization_EWK 155934 0.0004806294468683465
WpWpJJ_EWK 11047 0.010441219730054328
WpWpJJ_QCD 850 0.00991417828728171
WLLJJ_EWK 8474 0.0009769498668866026
WZTo3LNu_0Jets_MLL_4to50 no events
WZTo3LNu_1Jets_MLL_4to50 

In [ ]:
data_list_training